### IR Wavefunction

$$
\newcommand{\ket}[1]{\vert{#1}\rangle}\\
\newcommand{\bra}[1]{\langle{#1}\vert}\\
	\ket{\text{IR}} = \frac{1}{2}\bigg[\ket{\uparrow^d}\frac{1}{\sqrt 2}\left(\ket{k_1 \downarrow} + \ket{k_{-1} \downarrow}\right) - \ket{\downarrow^d}\frac{1}{\sqrt 2}\left(\ket{k_1 \uparrow} + \ket{k_{-1} \uparrow}\right) + \ket{2^d}\ket{0} - \ket{0^d}\ket{k_1 \uparrow, k_1 \downarrow,k_{-1} \uparrow,k_{-1} \downarrow}\bigg] \otimes \\\ket{k_{-2} \uparrow} \ket{k_{-2} \downarrow} \ket{k_{-3} \uparrow} \ket{k_{-3} \downarrow} 
$$

### Creating impurity and c-bath basis

In [7]:
from qutip import *
from math import *
import itertools

moms = list(range(-2, 3))
moms.remove(0)
print (moms)
spins = ("u", "d")
occs = (0, 1)
cb = dict()
for k, spin, occ in itertools.product(moms, spins, occs):
    cb[str(k)+spin+str(occ)] = basis(2,occ)
ib = dict()
for spin, occ in itertools.product(spins, occs):
    ib[spin+str(occ)] = basis(2,occ)

[-2, -1, 1, 2]


### Writing the IR wavefunction

In [8]:
updn = (1/sqrt(2)) * (tensor(ib["u1"], ib["d0"], cb["-1u0"], cb["-1d0"], cb["1u0"], cb["1d1"]) + tensor(ib["u1"], ib["d0"], cb["-1u0"], cb["-1d1"], cb["1u0"], cb["1d0"]))
dnup = (1/sqrt(2)) * (tensor(ib["u0"], ib["d1"], cb["-1u0"], cb["-1d0"], cb["1u1"], cb["1d0"]) + tensor(ib["u0"], ib["d1"], cb["-1u1"], cb["-1d0"], cb["1u0"], cb["1d0"]))
# cloud_part = tensor(ib["u1"], kcloud_dn) - tensor(ib["d1"], kcloud_up) + tensor(ib["u1"], ib["d1"],kcloud_dn) + tensor(imp_2, kcloud_0) - tensor(imp_0, kcloud_2)
ioms1 = tensor(cb["-2u1"], cb["-2d1"])
ioms2 = tensor(cb["2u0"], cb["2d0"])
psi_IR = tensor(ioms1, updn - dnup, ioms2).unit()

In [ ]:
c1 = 1
c2 = 1
c3 = 1
V = 1
J = 1
K = 1
eta21_u = V * tensor(create(2), identity(2), destroy(2), c1 * num(2) + c2 * (1 - num(2)), identity(2), identity(2)) + c3 * ((J/2) + (K/2)) 
Udag_21_u = (1/sqrt(2))(1 + eta_21_u)
Udag_21_d = (1/sqrt(2))(1 + eta_21_d)